In [2]:
!pip install selenium

In [3]:
!pip install chromedriver-binary

In [4]:
!brew install chromedriver

Updating Homebrew...
^C


In [ ]:
# append import path
import os, sys
print(os.getcwd())
sys.path.append('../')

In [8]:
import os, sys
from model  import estimation_silent
import time
from selenium import webdriver



driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
# driver.get('https://ikujipapa.net/archives/16621')
driver.get('https://www.google.com')
time.sleep(3)
result = driver.find_elements_by_tag_name('p')
result_parents = set()
for r in result:
    result_parents.add(r.find_element_by_xpath('..'))

for r in result_parents:
    print(r.get_attribute("id"))
    
print([r.text for r in result])
driver.quit()

[]


In [ ]:
import time
from selenium import webdriver

driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
driver.get('https://ikujipapa.net/archives/16621')
results_raw = driver.find_elements_by_tag_name('p')


for r in results_raw:
    print(r.get_attribute("id"))
    
print([r.text for r in results])
driver.quit()

In [1]:
MODEL_PATH = '../data/processing/20190526_名詞のみ.bin'
CONTENT = 'ここにテキストが入ります。'

# -*- coding: utf-8 -*-

import sys
import subprocess as cmd
import MeCab
import fasttext as ft
from sklearn.metrics import mean_squared_error


def text2bow(obj, mod):

    # input: ファイルの場合はmod="file", input: 文字列の場合はmod="str"

    """
    if mod == "file":
        morp = cmd.getstatusoutput("cat " + obj + " | mecab -Owakati")
    elif mod == "str":
        morp = cmd.getstatusoutput("echo " + obj + " | mecab -Owakati")
    else:
        print ("error!!")
        sys.exit(0)
    """

    mecab = MeCab.Tagger("-Ochasen")
    morp = mecab.parse(obj)

    # print(morp)

    words = morp[1]
    words = words.replace('\n','')

    return words

def Scoring(prob):

    score = 0.0
    for e in prob.keys():
        score += e*prob[e]

    return score


def SentimentEstimation(input_txt, clf):

    prob = {}

    bow = text2bow(input_txt, mod="str")

    # print(bow)
    estimate = clf.predict_proba(texts=[bow], k=5)[0]

    for e in estimate:
        index = int(e[0][9])
        prob[index] = e[1]

    score = Scoring(prob)
    return score


def main(model):

    score = SentimentEstimation(CONTENT, model)
    print(score)


if __name__ == "__main__":

    argvs = sys.argv

    _usage = """--                                                                                                                                                       
Usage:                                                                                                                                                                   
    python estimation.py [model]                                                                                                                                         
Args:                                                                                                                                                                    
    [model]: The argument is a model for sentiment estimation that is trained by fastText.                                                                               
""".rstrip()

    if len(argvs) < 2:
        print (_usage)
        sys.exit(0)

    model = ft.load_model(MODEL_PATH)

    main(model)

2.888675


NameError: name 'argvs' is not defined